# AES encryption on each file and saving the data and key to one file

In [1]:
import os
import csv
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad
from time import perf_counter

def extract_user_id(filename):
    parts = filename.split('_')
    if len(parts) > 1 and parts[0] == 'user':
        return parts[1]
    return 'Unknown' 

def encrypt_file(file_path, encrypted_folder, keys_folder, encryption_report):
    start_time = perf_counter()

    key = get_random_bytes(32)  # AES-256
    iv = get_random_bytes(AES.block_size)

    with open(file_path, 'rb') as f:
        data = f.read()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted_data = cipher.encrypt(pad(data, AES.block_size))

    base_filename = os.path.basename(file_path)
    encrypted_file_path = os.path.join(encrypted_folder, f"encrypted_{base_filename}")
    with open(encrypted_file_path, 'wb') as f:
        f.write(encrypted_data)

    key_iv_file_path = os.path.join(keys_folder, f"key_iv_{base_filename}.bin")
    with open(key_iv_file_path, 'wb') as f:
        f.write(key + iv)  

    end_time = perf_counter()
    encryption_duration = end_time - start_time

    user_id = extract_user_id(base_filename)
    encryption_report.append({
        'filename': base_filename,
        'user_id': user_id,
        'encryption_duration': encryption_duration
    })

    return encrypted_file_path, key_iv_file_path

def write_encryption_report(encryption_report, report_path):
    report_file_path = os.path.join(report_path, "AES_encryption_report.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['filename', 'user_id', 'encryption_duration'])
        writer.writeheader()
        for entry in encryption_report:
            writer.writerow(entry)

def encrypt_all_files_in_folder(source_folder, encrypted_folder, keys_folder, encryption_report):
    # Ensure the target folders exist
    if not os.path.exists(encrypted_folder):
        os.makedirs(encrypted_folder)
    if not os.path.exists(keys_folder):
        os.makedirs(keys_folder)

    # Encrypt all files in the source folder
    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, encrypted_folder, keys_folder, encryption_report)
            print(f"Encrypted {filename}")

def main():
    source_folder = input("Enter the path to the folder containing aggregated files: ")
    encrypted_folder = input("Enter the path to the folder where encrypted files will be saved: ")
    keys_folder = input("Enter the path to the folder where encryption keys will be stored: ")
    report_folder = input("Enter the path to the folder where the encryption report will be saved: ")

    encryption_report = []
    encrypt_all_files_in_folder(source_folder, encrypted_folder, keys_folder, encryption_report)
    write_encryption_report(encryption_report, report_folder)
    print("Encryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing aggregated files: C:/Users/ROHAN/IotSimulation/DecForAggreator
Enter the path to the folder where encrypted files will be saved: C:/Users/ROHAN/IotSimulation/AES_enc
Enter the path to the folder where encryption keys will be stored: C:/Users/ROHAN/IotSimulation/AES_Keys
Enter the path to the folder where the encryption report will be saved: C:/Users/ROHAN/IotSimulation
Encrypted user_10_aggregated.txt
Encrypted user_1_aggregated.txt
Encrypted user_2_aggregated.txt
Encrypted user_3_aggregated.txt
Encrypted user_4_aggregated.txt
Encrypted user_5_aggregated.txt
Encrypted user_6_aggregated.txt
Encrypted user_7_aggregated.txt
Encrypted user_8_aggregated.txt
Encrypted user_9_aggregated.txt
Encryption process completed. Report generated.


In [ ]:
C:/Users/ROHAN/IotSimulation/DecForAggreator

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_enc

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_Keys

# AES Decryption of File

In [1]:
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

def decrypt_file(encrypted_file_path, key_iv_path, decrypted_folder):
    # Read the key and IV from the key file
    with open(key_iv_path, 'rb') as key_iv_file:
        key = key_iv_file.read(32)  # AES-256 key size is 32 bytes
        iv = key_iv_file.read(16)  # AES block size is 16 bytes

    # Read the encrypted data
    with open(encrypted_file_path, 'rb') as encrypted_file:
        encrypted_data = encrypted_file.read()

    # Create AES cipher and decrypt data
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)

    # Save the decrypted data to a new file
    base_filename = os.path.basename(encrypted_file_path)
    decrypted_filename = base_filename.replace("encrypted_", "decrypted_")
    decrypted_file_path = os.path.join(decrypted_folder, decrypted_filename)

    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    return decrypted_file_path

def decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder):
    # Ensure the decrypted folder exists
    if not os.path.exists(decrypted_folder):
        os.makedirs(decrypted_folder)

    # Decrypt all files in the encrypted folder
    for filename in os.listdir(encrypted_folder):
        if filename.startswith("encrypted_"):
            encrypted_file_path = os.path.join(encrypted_folder, filename)
            # Adjust the filename format to match your key file naming convention
            key_iv_filename = f"key_iv_{filename.replace('encrypted_', '')}.bin"
            key_iv_path = os.path.join(keys_folder, key_iv_filename)
            decrypted_file_path = decrypt_file(encrypted_file_path, key_iv_path, decrypted_folder)
            print(f"Decrypted {filename} to {decrypted_file_path}")

def main():
    encrypted_folder = input("Enter the path to the folder containing encrypted files: ")
    keys_folder = input("Enter the path to the folder where encryption keys are stored: ")
    decrypted_folder = input("Enter the path to the folder where decrypted files will be saved: ")

    decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder)
    print("Decryption process completed.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing encrypted files: C:/Users/ROHAN/IotSimulation/NoBase64
Enter the path to the folder where encryption keys are stored: C:/Users/ROHAN/IotSimulation/AES_Keys
Enter the path to the folder where decrypted files will be saved: C:/Users/ROHAN/IotSimulation/AES-DecFile
Decryption process completed.


In [ ]:
C:/Users/ROHAN/IotSimulation/AES_enc

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_Keys

In [ ]:
C:/Users/ROHAN/IotSimulation/AES-DecFile

In [5]:
import os
import time
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import csv

def decrypt_file(encrypted_file_path, key_iv_path, decrypted_folder):
    start_time = time.perf_counter()

    with open(key_iv_path, 'rb') as key_iv_file:
        key = key_iv_file.read(32)
        iv = key_iv_file.read(16)

    with open(encrypted_file_path, 'rb') as encrypted_file:
        encrypted_data = encrypted_file.read()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)

    base_filename = os.path.basename(encrypted_file_path)
    decrypted_filename = base_filename.replace("encrypted_", "decrypted_")
    decrypted_file_path = os.path.join(decrypted_folder, decrypted_filename)

    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    end_time = time.perf_counter()
    decryption_time = end_time - start_time

    parts = base_filename.split('_')
    user_id_index = parts.index('user') + 1 if 'user' in parts else None
    user_id = parts[user_id_index] if user_id_index is not None else 'Unknown'

    return decrypted_file_path, user_id, decryption_time

def decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder, report_directory):
    if not os.path.exists(decrypted_folder):
        os.makedirs(decrypted_folder)

    decryption_report = []

    for filename in os.listdir(encrypted_folder):
        if filename.startswith("encrypted_"):
            encrypted_file_path = os.path.join(encrypted_folder, filename)
            key_iv_filename = f"key_iv_{filename.replace('encrypted_', '')}.bin"
            key_iv_path = os.path.join(keys_folder, key_iv_filename)
            decrypted_file_path, user_id, decryption_time = decrypt_file(encrypted_file_path, key_iv_path, decrypted_folder)
            print(f"Decrypted {filename} to {decrypted_file_path}")

            decryption_report.append({
                'user_id': user_id,
                'filename': filename,
                'decryption_time': decryption_time
            })

    report_file_name = "AES-DecReport.csv"
    report_path = os.path.join(report_directory, report_file_name)

    with open(report_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['user_id', 'filename', 'decryption_time'])
        writer.writeheader()
        for entry in decryption_report:
            writer.writerow(entry)

def main():
    encrypted_folder = input("Enter the path to the folder containing encrypted files: ")
    keys_folder = input("Enter the path to the folder where encryption keys are stored: ")
    decrypted_folder = input("Enter the path to the folder where decrypted files will be saved: ")
    report_directory = input("Enter the directory where the decryption report will be saved: ")

    decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder, report_directory)
    print("Decryption process completed.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing encrypted files: C:/Users/ROHAN/IotSimulation/AES_enc
Enter the path to the folder where encryption keys are stored: C:/Users/ROHAN/IotSimulation/AES_Keys
Enter the path to the folder where decrypted files will be saved: C:/Users/ROHAN/IotSimulation/AES-DecFile
Enter the directory where the decryption report will be saved: C:/Users/ROHAN/IotSimulation
Decrypted encrypted_user_10_auth_EMP009_EMP002_merged_1.txt to C:/Users/ROHAN/IotSimulation/AES-DecFile\decrypted_user_10_auth_EMP009_EMP002_merged_1.txt
Decrypted encrypted_user_11_auth_EMP007_EMP020_EMP017_merged_1.txt to C:/Users/ROHAN/IotSimulation/AES-DecFile\decrypted_user_11_auth_EMP007_EMP020_EMP017_merged_1.txt
Decrypted encrypted_user_12_auth_EMP011_EMP008_EMP016_merged_1.txt to C:/Users/ROHAN/IotSimulation/AES-DecFile\decrypted_user_12_auth_EMP011_EMP008_EMP016_merged_1.txt
Decrypted encrypted_user_13_auth_EMP013_EMP002_merged_1.txt to C:/Users/ROHAN/IotSimulation/AES-DecFile\decrypted_us

# RE-encryption Time

In [4]:
import os
import csv
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from time import perf_counter

def extract_user_id(filename):
    parts = filename.split('_')
    if 'user' in parts:
        user_id_index = parts.index('user') + 1
        if user_id_index < len(parts):
            return parts[user_id_index]
    return 'Unknown'  

def decrypt_file(encrypted_file_path, key_iv_path):
    start_decrypt = perf_counter()

    with open(key_iv_path, 'rb') as key_iv_file:
        key = key_iv_file.read(32)
        iv = key_iv_file.read(16)

    with open(encrypted_file_path, 'rb') as encrypted_file:
        encrypted_data = encrypted_file.read()

    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)

    end_decrypt = perf_counter()
    decryption_time = end_decrypt - start_decrypt

    return decrypted_data, decryption_time

def re_encrypt_data(decrypted_data):
    start_encrypt = perf_counter()

    new_key = get_random_bytes(32)  
    new_iv = get_random_bytes(AES.block_size)
    cipher = AES.new(new_key, AES.MODE_CBC, new_iv)
    re_encrypted_data = cipher.encrypt(pad(decrypted_data, AES.block_size))

    end_encrypt = perf_counter()
    re_encryption_time = end_encrypt - start_encrypt

    return re_encrypted_data, new_key, new_iv, re_encryption_time

def process_files(encrypted_folder, keys_folder, re_encrypted_folder, re_encryption_report):
    for filename in os.listdir(encrypted_folder):
        if filename.startswith("encrypted_user_"):
            encrypted_file_path = os.path.join(encrypted_folder, filename)
            key_iv_filename = f"key_iv_{filename.replace('encrypted_', '')}.bin"
            key_iv_path = os.path.join(keys_folder, key_iv_filename)

            decrypted_data, decryption_time = decrypt_file(encrypted_file_path, key_iv_path)
            re_encrypted_data, new_key, new_iv, re_encryption_time = re_encrypt_data(decrypted_data)

            re_encrypted_file_path = os.path.join(re_encrypted_folder, f"re_encrypted_{filename}")
            with open(re_encrypted_file_path, 'wb') as f:
                f.write(re_encrypted_data)

            new_key_iv_file_path = os.path.join(re_encrypted_folder, f"new_key_iv_{filename}.bin")
            with open(new_key_iv_file_path, 'wb') as f:
                f.write(new_key + new_iv)

            user_id = extract_user_id(filename)
            re_encryption_report.append({
                'user_id': user_id,
                're_encryption_time': re_encryption_time,
                'decryption_time': decryption_time
            })

def write_report(re_encryption_report, report_path):
    report_file_path = os.path.join(report_path, "Re-encryptionAES.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['user_id', 're_encryption_time', 'decryption_time'])
        writer.writeheader()
        for entry in re_encryption_report:
            writer.writerow(entry)

def main():
    encrypted_folder = input("Enter the path to the folder containing encrypted files: ")
    keys_folder = input("Enter the path to the folder where encryption keys are stored: ")
    re_encrypted_folder = input("Enter the path to the folder where re-encrypted files will be saved: ")
    report_folder = input("Enter the path to the folder where the re-encryption report will be saved: ")

    if not os.path.exists(re_encrypted_folder):
        os.makedirs(re_encrypted_folder)

    re_encryption_report = []
    process_files(encrypted_folder, keys_folder, re_encrypted_folder, re_encryption_report)
    write_report(re_encryption_report, report_folder)

    print("Re-encryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing encrypted files: C:/Users/ROHAN/IotSimulation/AES_enc
Enter the path to the folder where encryption keys are stored: C:/Users/ROHAN/IotSimulation/AES_Keys
Enter the path to the folder where re-encrypted files will be saved: C:/Users/ROHAN/IotSimulation/Re_AES-Encryption/File
Enter the path to the folder where the re-encryption report will be saved: C:/Users/ROHAN/IotSimulation/Re_AES-Encryption
Re-encryption process completed. Report generated.


In [ ]:
C:/Users/ROHAN/IotSimulation/AES_enc

In [ ]:
C:/Users/ROHAN/IotSimulation/AES_Keys

In [ ]:
C:/Users/ROHAN/IotSimulation/Re_AES-Encryption/File

In [ ]:
C:/Users/ROHAN/IotSimulation/Re_AES-Encryption/key

# AES ENCRYPTION FOR CONFERENCE

In [1]:
import os
import csv
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import time

def generate_aes_key():
    return os.urandom(32)  # 256-bit key

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)  # IV for AES

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext) + padder.finalize()

    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return iv + ciphertext

def encrypt_file(file_path, encrypted_folder, keys_folder, encryption_report):
    start_time = time.perf_counter()

    with open(file_path, 'rb') as f:
        plaintext = f.read()

    try:
        file_data = json.loads(plaintext.decode())
        if isinstance(file_data, list) and file_data:
            user_id = file_data[0].get('user_id', 'Unknown')
            authorized_users = file_data[0].get('authorized_data_users', [])
        else:
            user_id = 'Unknown'
            authorized_users = []
    except json.JSONDecodeError:
        user_id = 'Unknown'
        authorized_users = []

    authorized_users_str = '_'.join(authorized_users)

    key = generate_aes_key()
    encrypted_data = aes_encrypt(key, plaintext)

    base_filename = os.path.basename(file_path).replace('aggregated', f'auth_{authorized_users_str}_aggregated')
    encrypted_file_path = os.path.join(encrypted_folder, f"encrypted_{base_filename}")
    with open(encrypted_file_path, 'wb') as f:
        f.write(encrypted_data)

    key_file_path = os.path.join(keys_folder, f"key_{base_filename}.bin")
    with open(key_file_path, 'wb') as f:
        f.write(key)

    end_time = time.perf_counter()
    encryption_duration = end_time - start_time
    encrypted_file_size = os.path.getsize(encrypted_file_path)

    encryption_report.append({
        'filename': base_filename,
        'user_id': user_id,
        'authorized_users': ','.join(authorized_users),
        'encryption_duration': encryption_duration,
        'encrypted_file_size': encrypted_file_size
    })

def write_encryption_report(encryption_report, report_path):
    report_file_path = os.path.join(report_path, "AES_encryption_report.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['filename', 'user_id', 'authorized_users', 'encryption_duration', 'encrypted_file_size'])
        writer.writeheader()
        for entry in encryption_report:
            writer.writerow(entry)

def encrypt_all_files_in_folder(source_folder, encrypted_folder, keys_folder, encryption_report):
    if not os.path.exists(encrypted_folder):
        os.makedirs(encrypted_folder)
    if not os.path.exists(keys_folder):
        os.makedirs(keys_folder)

    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, encrypted_folder, keys_folder, encryption_report)
            print(f"Encrypted {filename}")

def main():
    source_folder = input("Enter the path to the folder containing files to encrypt: ")
    encrypted_folder = input("Enter the path to the folder where encrypted files will be saved: ")
    keys_folder = input("Enter the path to the folder where encryption keys will be stored: ")
    report_folder = input("Enter the path to the folder where the encryption report will be saved: ")

    encryption_report = []
    encrypt_all_files_in_folder(source_folder, encrypted_folder, keys_folder, encryption_report)
    write_encryption_report(encryption_report, report_folder)
    print("Encryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing files to encrypt: C:/Users/ROHAN/IotSimulation/ConReq/AggData50kb
Enter the path to the folder where encrypted files will be saved: C:/Users/ROHAN/IotSimulation/ConReq/AESCipherText
Enter the path to the folder where encryption keys will be stored: C:/Users/ROHAN/IotSimulation/ConReq/AESCipherKey
Enter the path to the folder where the encryption report will be saved: C:/Users/ROHAN/IotSimulation/ConReq
Encrypted user_100_aggregated.txt
Encrypted user_101_aggregated.txt
Encrypted user_102_aggregated.txt
Encrypted user_103_aggregated.txt
Encrypted user_104_aggregated.txt
Encrypted user_105_aggregated.txt
Encrypted user_106_aggregated.txt
Encrypted user_107_aggregated.txt
Encrypted user_108_aggregated.txt
Encrypted user_109_aggregated.txt
Encrypted user_10_aggregated.txt
Encrypted user_110_aggregated.txt
Encrypted user_111_aggregated.txt
Encrypted user_112_aggregated.txt
Encrypted user_113_aggregated.txt
Encrypted user_114_aggregated.txt
Encrypted u

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850kbTrials

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850KB_AES_Enc

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850KB_AES_Enc_key

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AggData50kb

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AESCipherText

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AESCipherKey

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq

# AES Decryption and Report

In [2]:
import os
import csv
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import time

def aes_decrypt(key, iv, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    return plaintext

def decrypt_file(encrypted_file_path, key_file_path, decrypted_folder, decryption_report):
    start_time = time.perf_counter()

    with open(key_file_path, 'rb') as f:
        key = f.read(32)  # AES 256-bit key

    with open(encrypted_file_path, 'rb') as f:
        encrypted_data = f.read()

    iv = encrypted_data[:16]
    ciphertext = encrypted_data[16:]

    if len(iv) != 16:
        raise ValueError("Invalid IV size for CBC mode.")

    decrypted_data = aes_decrypt(key, iv, ciphertext)

    base_filename = os.path.basename(encrypted_file_path).replace('encrypted_', '')
    decrypted_file_path = os.path.join(decrypted_folder, base_filename)
    with open(decrypted_file_path, 'wb') as f:
        f.write(decrypted_data)

    end_time = time.perf_counter()
    decryption_duration = end_time - start_time

    user_id = base_filename.split('_')[1]  # Assuming filename format: user_{user_id}_...

    decryption_report.append({
        'filename': base_filename,
        'user_id': user_id,
        'decryption_duration': decryption_duration
    })

def write_decryption_report(decryption_report, report_path):
    report_file_path = os.path.join(report_path, "AES_decryption_report.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['filename', 'user_id', 'decryption_duration'])
        writer.writeheader()
        for entry in decryption_report:
            writer.writerow(entry)

def decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder, decryption_report):
    if not os.path.exists(decrypted_folder):
        os.makedirs(decrypted_folder)

    for filename in os.listdir(encrypted_folder):
        if filename.startswith("encrypted_"):
            encrypted_file_path = os.path.join(encrypted_folder, filename)
            key_file_path = os.path.join(keys_folder, f"key_{filename.replace('encrypted_', '')}.bin")
            decrypt_file(encrypted_file_path, key_file_path, decrypted_folder, decryption_report)
            print(f"Decrypted {filename}")

def main():
    encrypted_folder = input("Enter the path to the folder containing encrypted files: ")
    keys_folder = input("Enter the path to the folder where encryption keys are stored: ")
    decrypted_folder = input("Enter the path to the folder where decrypted files will be saved: ")
    report_folder = input("Enter the path to the folder where the decryption report will be saved: ")

    decryption_report = []
    decrypt_all_files_in_folder(encrypted_folder, keys_folder, decrypted_folder, decryption_report)
    write_decryption_report(decryption_report, report_folder)
    print("Decryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing encrypted files: C:/Users/ROHAN/IotSimulation/ConReq/AESCipherText
Enter the path to the folder where encryption keys are stored: C:/Users/ROHAN/IotSimulation/ConReq/AESCipherKey
Enter the path to the folder where decrypted files will be saved: C:/Users/ROHAN/IotSimulation/ConReq/AESDecData
Enter the path to the folder where the decryption report will be saved: C:/Users/ROHAN/IotSimulation/ConReq
Decrypted encrypted_user_100_auth_EMP080_EMP069_aggregated.txt
Decrypted encrypted_user_101_auth_EMP035_EMP053_aggregated.txt
Decrypted encrypted_user_102_auth_EMP027_EMP079_EMP042_EMP046_EMP014_EMP024_EMP082_EMP048_aggregated.txt
Decrypted encrypted_user_103_auth_EMP068_EMP075_EMP031_EMP021_EMP011_EMP029_EMP004_EMP063_EMP001_aggregated.txt
Decrypted encrypted_user_104_auth_EMP075_aggregated.txt
Decrypted encrypted_user_105_auth_EMP025_EMP066_EMP026_EMP061_EMP019_EMP047_EMP024_EMP073_EMP060_EMP031_aggregated.txt
Decrypted encrypted_user_106_auth_EMP041_E

Decrypted encrypted_user_182_auth_EMP030_aggregated.txt
Decrypted encrypted_user_183_auth_EMP029_EMP078_EMP061_EMP052_EMP035_EMP063_EMP019_EMP011_EMP048_EMP021_aggregated.txt
Decrypted encrypted_user_184_auth_EMP017_EMP007_EMP025_EMP076_EMP068_aggregated.txt
Decrypted encrypted_user_185_auth_EMP039_EMP052_EMP002_EMP041_EMP059_EMP055_EMP021_aggregated.txt
Decrypted encrypted_user_186_auth_EMP020_EMP037_EMP082_EMP074_EMP018_EMP046_EMP057_EMP014_EMP045_aggregated.txt
Decrypted encrypted_user_187_auth_EMP057_EMP003_EMP021_EMP032_EMP046_EMP064_EMP030_EMP050_aggregated.txt
Decrypted encrypted_user_188_auth_EMP049_EMP034_EMP015_EMP017_EMP067_EMP080_EMP048_EMP004_aggregated.txt
Decrypted encrypted_user_189_auth_EMP057_EMP028_EMP044_EMP031_aggregated.txt
Decrypted encrypted_user_18_auth_EMP043_EMP074_EMP010_EMP033_EMP052_EMP038_aggregated.txt
Decrypted encrypted_user_190_auth_EMP083_EMP006_EMP053_EMP080_EMP068_EMP058_EMP064_EMP041_EMP061_aggregated.txt
Decrypted encrypted_user_191_auth_EMP063_E

Decrypted encrypted_user_92_auth_EMP041_EMP061_EMP066_aggregated.txt
Decrypted encrypted_user_93_auth_EMP069_aggregated.txt
Decrypted encrypted_user_94_auth_EMP047_EMP064_EMP042_EMP027_aggregated.txt
Decrypted encrypted_user_95_auth_EMP071_EMP068_EMP073_EMP077_EMP019_EMP013_aggregated.txt
Decrypted encrypted_user_96_auth_EMP022_EMP068_EMP005_EMP011_EMP008_EMP063_EMP049_EMP002_EMP048_aggregated.txt
Decrypted encrypted_user_97_auth_EMP032_EMP001_EMP035_EMP048_EMP027_EMP044_EMP005_EMP053_aggregated.txt
Decrypted encrypted_user_98_auth_EMP025_EMP027_EMP079_EMP038_EMP075_EMP039_EMP058_EMP035_EMP041_EMP008_aggregated.txt
Decrypted encrypted_user_99_auth_EMP082_aggregated.txt
Decrypted encrypted_user_9_auth_EMP069_EMP028_EMP031_EMP012_EMP071_EMP032_EMP070_EMP037_aggregated.txt
Decryption process completed. Report generated.


In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850KB_AES_Enc

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850KB_AES_Enc_key

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/850KB_AES_Dec

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AESCipherText

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AESCipherKey

In [ ]:
C:/Users/ROHAN/IotSimulation/ConReq/AESDecData

In [ ]:
AESDecData